In [342]:
import pandas as pd
import numpy as np
import itertools
import os
import time
from multiprocessing import Pool
from functools import partial

In [343]:
def checkmeshPool(pp):
    points, meshes = pp
    n = len(meshes.values)
    for b in range(n):
        bb = meshes.values[b]
        if not bb_intersection_over_union(points,bb):
            return (points[6], True )
    return (points[6], False )


In [344]:
def log(message):
    print('{},{}'.format(time.time(), message))

In [345]:

#Reading all data csv
pov_ = pd.read_csv(r"C:\Users\derep003\Documents\source\sightline\pov_.csv",header=None )
pov_.columns = ["x","y","z" ]
pov_.head()


,x,y,z
0,17.120115,-10.690694,19.879354
1,17.120115,-10.690694,19.879354


In [346]:
target_ = pd.read_csv(r"C:\Users\derep003\Documents\source\sightline\targets_.csv",header=None )
target_.columns = ["x1","y1","z1" ]
target_.head()

,x1,y1,z1
0,6.495480,18.018316,5.849932
1,6.495480,18.018316,11.128035
2,14.890557,18.018316,5.849932
3,14.890557,18.018316,11.128035
4,14.890557,18.018316,5.849932


In [347]:
meshes_ = pd.read_csv(r"C:\Users\derep003\Documents\source\sightline\context_.csv",header=None, index_col=0  )
meshes_.columns = ["xMax","yMax","zMax","xMin","yMin","zMin","id" ]
meshes_.head()

,xMax,yMax,zMax,xMin,yMin,zMin,id
0,,,,,,,
0,-8.9195,-12.3862,4.4770,3.5608,8.3406,22.7579,77e27c02-05fd-49c5-b2b0-80e8c46f3284
1,0.6135,-12.3862,4.4770,13.0938,8.3406,22.7579,77b32997-5eb1-40e6-92e9-072ecf7a4545
2,6.4955,18.0183,5.8499,14.8906,25.5255,11.1280,12b88893-4efd-4525-a226-9fcd1190b12a
0,-8.9195,-12.3862,4.4770,3.5608,8.3406,22.7579,77e27c02-05fd-49c5-b2b0-80e8c46f3284
1,0.6135,-12.3862,4.4770,13.0938,8.3406,22.7579,77b32997-5eb1-40e6-92e9-072ecf7a4545


In [348]:
mesh_faces = pd.read_csv(r"C:\Users\derep003\Documents\source\sightline\mesh_faces.csv",header=None  )
mesh_faces.columns = ["xMax","yMax","zMax","xMin","yMin","zMin", "id" ]
mesh_faces.head()

,xMax,yMax,zMax,xMin,yMin,zMin,id
0,-0.1165,-0.7225,5.7024,-0.0402,-0.6582,5.7540,77e27c02-05fd-49c5-b2b0-80e8c46f3284
1,-0.0844,-0.7397,5.5804,0.0683,-0.7148,5.7024,77e27c02-05fd-49c5-b2b0-80e8c46f3284
2,-0.1867,-0.8056,5.5942,-0.1415,-0.6641,5.7535,77e27c02-05fd-49c5-b2b0-80e8c46f3284
3,-0.1478,-0.8151,5.5479,-0.0844,-0.7148,5.5942,77e27c02-05fd-49c5-b2b0-80e8c46f3284
4,-0.2813,-0.7401,5.7535,-0.1708,-0.6377,5.8508,77e27c02-05fd-49c5-b2b0-80e8c46f3284


In [349]:
pov_['_tmpkey'] = 1
target_['_tmpkey'] = 1

lines = pd.merge(pov_, target_, on='_tmpkey').drop('_tmpkey', axis=1)
lines.index = pd.MultiIndex.from_product((pov_.index, target_.index))

pov_.drop('_tmpkey', axis=1, inplace=True)
target_.drop('_tmpkey', axis=1, inplace=True)

lines = pd.DataFrame(lines, columns =['x','y','z','x1','y1','z1'] ).reset_index()

lines = lines.drop(columns=["level_0", "level_1"])
lines.head()

,x,y,z,x1,y1,z1
0,17.120115,-10.690694,19.879354,6.495480,18.018316,5.849932
1,17.120115,-10.690694,19.879354,6.495480,18.018316,11.128035
2,17.120115,-10.690694,19.879354,14.890557,18.018316,5.849932
3,17.120115,-10.690694,19.879354,14.890557,18.018316,11.128035
4,17.120115,-10.690694,19.879354,14.890557,18.018316,5.849932


In [350]:
bbx = pd.DataFrame(columns = ["xMax","yMax","zMax","xMin","yMin","zMin",'order'])
bbx['xMax'] = lines[['x', 'x1']].values.max(1)
bbx['yMax'] = lines[['y', 'y1']].values.max(1)
bbx['zMax'] = lines[['z', 'z1']].values.max(1)
bbx['xMin'] = lines[['x', 'x1']].values.min(1)
bbx['yMin'] = lines[['y', 'y1']].values.min(1)
bbx['zMin'] = lines[['z', 'z1']].values.min(1)
bbx['order'] = bbx.index

bbx.head()

,xMax,yMax,zMax,xMin,yMin,zMin,order
0,17.120115,18.018316,19.879354,6.495480,-10.690694,5.849932,0
1,17.120115,18.018316,19.879354,6.495480,-10.690694,11.128035,1
2,17.120115,18.018316,19.879354,14.890557,-10.690694,5.849932,2
3,17.120115,18.018316,19.879354,14.890557,-10.690694,11.128035,3
4,17.120115,18.018316,19.879354,14.890557,-10.690694,5.849932,4


In [351]:
log('finish creating bbx dt {}'.format(len(bbx)))
log('possible calculations {}'.format(len(bbx)* len(meshes_)))

1569625108.4521687,finish creating bbx dt 288180
1569625108.4521687,possible calculations 1729080


In [352]:
def checkmesh(meshes, points):
    global count
    n = len(meshes)
    for b in range(n):
        bb = meshes[b]
        if  bb_intersection_over_union(points,bb):
            
            return True

    return False 

In [353]:
def bb_intersection_over_union(boxA, boxB):
 
#     interArea = np.any([np.greater(boxA[0],boxB[3]),np.greater( boxA[3],boxB[0]) 
#     ,np.greater(boxA[1],boxB[4]), np.greater(boxA[4],boxB[1]) 
#     ,np.greater(boxA[2],boxB[5]), np.greater(boxA[5], boxB[2])]) 

    interArea =  ((boxA[0] > boxB[3]) or ( boxA[3] > boxB[0]) or (boxA[1] >boxB[4]) or (boxA[4]> boxB[1]) 
    or (boxA[2] > boxB[5]) or (boxA[5] > boxB[2])) 

    return interArea

In [354]:
result = bbx.apply(lambda x: checkmesh( meshes_.values, x), axis=1)
bbx= bbx.assign(hits=pd.Series(result).values)
bbx[bbx["hits"] == True].head()

,xMax,yMax,zMax,xMin,yMin,zMin,order,hits
0,17.120115,18.018316,19.879354,6.495480,-10.690694,5.849932,0,True
1,17.120115,18.018316,19.879354,6.495480,-10.690694,11.128035,1,True
2,17.120115,18.018316,19.879354,14.890557,-10.690694,5.849932,2,True
3,17.120115,18.018316,19.879354,14.890557,-10.690694,11.128035,3,True
4,17.120115,18.018316,19.879354,14.890557,-10.690694,5.849932,4,True
